# Setup, Pre-processing

In [1]:
# !pip install gurobipy
import gurobipy as gb
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
ev_data = pd.read_csv("WA_Electric_Vehicle_Population_Data.csv", dtype={'Postal Code': str})
distance_data = pd.read_csv("distance.csv", dtype={'PostalCode1': str, 'PostalCode2': str})
# ev_data = pd.read_csv("/content/Electric_Vehicle_Population_Data.csv", dtype={'Postal Code': str})
# distance_data = pd.read_csv("/content/distance.csv", dtype={'PostalCode1': str, 'PostalCode2': str})

In [3]:
# Define coverage
def calculate_coverage(row, average_ranges, theta):
    postal1 = row['PostalCode1']
    postal2 = row['PostalCode2']
    distance = row['Distance']

    coverage1 = 1 if (theta * average_ranges.get(postal1, 0) >= distance) else 0
    coverage2 = 1 if (theta * average_ranges.get(postal2, 0) >= distance) else 0

    return pd.Series([coverage1, coverage2])

def calculate_C(distance_data):
    # C(i, j) = 1 if i is covered by j, otherwise C(i, j) = 0
    C = {}
    for _, row in distance_data.iterrows():
        C[(row['PostalCode1'], row['PostalCode2'])] = row['Coverage1']
        C[(row['PostalCode2'], row['PostalCode1'])] = row['Coverage2']

    # Ensure each location covers itself
    postal_codes = set(distance_data['PostalCode1']).union(distance_data['PostalCode2'])
    for postal_code in postal_codes:
        C[(postal_code, postal_code)] = 1

    return C

# Calculate the count of EVs in each zip code
EV_count = ev_data['Postal Code'].value_counts().to_dict()

# Calculate the average electric range for each zip code
average_range = ev_data.groupby('Postal Code')['Electric Range'].mean().to_dict()

# Calculate demand (EV_j) as the total EV range in each zip code
EV = {key: EV_count[key] * average_range[key] for key in EV_count if key in average_range}

distance_data[['Coverage1', 'Coverage2']] = distance_data.apply(
    calculate_coverage,
    axis=1,
    average_ranges=average_range,
    theta=0.2
)
C = calculate_C(distance_data)

# Base

In [6]:
############################### Initialize Model ###############################

model = gb.Model("EV_Charging_Optimization")
model.setParam('MIPGap', 1e-03)
model.Params.OutputFlag = 0


################################## Parameters ##################################

# Charging speeds in miles per hour
charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
environmental_impact = {2: 0.000027405, 3: 0.0002945}
H = 10  # Available hours per day
public_charger_demand = 0.3
avg_percentage_charged = 0.6
peak_charging_ev = 0.1

# Cost parameters
cost_per_level2_charger = 8250
cost_per_level3_charger = 65000
# Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
base_station_cost = 50000
# Electrical upgrade calculations
panel_capacity = 500 # amps
usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
existing_load = 200  # amps
remaining_capacity = usable_capacity - existing_load  # amps
upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers


############################## Decision Variables ##############################

x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")

demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")


################################## Objectives ##################################

total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
total_range = gb.quicksum((x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) for i in EV)

# Calculate the environmental impact based on demand fulfilled by each type
total_environmental_impact = (
    environmental_impact[2] * demand_fulfilled_by_level2 +
    environmental_impact[3] * demand_fulfilled_by_level3
)
#### Calculate the total cost
M = 10000
for i in EV:
    # Ensure only one tier is active for each location
    model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

    # Combined loads:
    combined_loads = x2[i] * 40 + x3[i] * 150

    # Constraints for each tier
    model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")

electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
chargers_setup_cost = gb.quicksum(
    x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
)
total_cost = chargers_setup_cost + station_setup_cost

# 1. Minimize the total cost
model.setObjectiveN(total_cost, index=0, priority=0)

# 2. Minimize Environmental impact
model.setObjectiveN(total_environmental_impact, index=1, priority=1)

# 3. Maximize total electric range per hour of charging
model.setObjectiveN(-total_range, index=2, priority=2)

model.ModelSense = GRB.MINIMIZE


################################## Constraints #################################
# EV at any location can reach at least one of the charging stations
for j in EV:
    model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

# Demand Coverage with Charging Capacity Requirement
for j in EV:
    total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
    model.addConstr(
        gb.quicksum(
            C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
            for i in EV
        ) >= total_demand_j,
        name=f"DemandCoverage_{j}"
    )

# Calculate the demand fulfilled by each type of charger
model.addConstr(
    demand_fulfilled_by_level2 == charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
    name="DemandFulfilledByLevel2"
)
model.addConstr(
    demand_fulfilled_by_level3 == charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
    name="DemandFulfilledByLevel3"
)

# # Simultaneous charging
# model.addConstr(gb.quicksum((x2[i] + x3[i]) for i in EV) / (sum(EV_count[j] for j in EV) * public_charger_demand) >= peak_charging_ev)
# for j in EV:
#     model.addConstr(gb.quicksum(C.get((i, j), 0) * (x2[i] + x3[i]) * z[i] for i in EV) >= EV_count[j] * public_charger_demand * peak_charging_ev)

# Linking Charging Stations and Chargers
M = 20  # Maximum number of charging stalls at each station
for i in EV:
    model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
    model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)


################################# Solve Model ##################################

model.optimize()

if model.Status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Charging Capacity per Hour: {total_range.getValue()}")
    print(f'Total Environmental Impact: {total_environmental_impact.getValue()}')
    print(f'Total Cost: {total_cost.getValue()}')
    print(f"Total Demand Fulfilled by Level 2 Chargers (miles): {demand_fulfilled_by_level2.x}")
    print(f"Total Demand Fulfilled by Level 3 Chargers (miles): {demand_fulfilled_by_level3.x}")
    print(f'Total number of Level 2 Charger: {sum(x2[i].x for i in EV)}')
    print(f'Total number of Level 3 Charger: {sum(x3[i].x for i in EV)}')
    print(f'Total number of Station: {sum(z[i].x for i in EV)}')
    print("\n")
    for i in EV:
        if z[i].x > 0.5:  # Using 0.5 to account for numerical issues
            level2_chargers = int(x2[i].x)
            level3_chargers = int(x3[i].x)
            print(f"Set up station at {i} with {level2_chargers} Level 2 chargers and {level3_chargers} Level 3 chargers")
else:
    print("No optimal solution found.")

Set parameter MIPGap to value 0.001
Optimal Solution Found:
Total Charging Capacity per Hour: 225270.0
Total Environmental Impact: 481.2880695
Total Cost: 119899500.0
Total Demand Fulfilled by Level 2 Chargers (miles): 681900.0
Total Demand Fulfilled by Level 3 Chargers (miles): 1570800.0
Total number of Level 2 Charger: 4546.0
Total number of Level 3 Charger: 748.0
Total number of Station: 265.0


Set up station at 98188.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98501.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98117.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98112.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98125.0 with 0 Level 2 chargers and 20 Level 3 chargers
Set up station at 98133.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98155.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98225.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up

## Save Results

In [ ]:
data = []
col = ['PostalCode', 'StationSetup', 'Level2Chargers', 'Level3Chargers']

for i in EV:
    if z[i].x > 0.5:
        level2_chargers = int(round(x2[i].x))
        level3_chargers = int(round(x3[i].x))
        data.append([i, 1, level2_chargers, level3_chargers])
    else:
        data.append([i, 0, 0, 0])

df = pd.DataFrame(data, columns=col)
df.to_csv('results.csv', index=False)

from google.colab import files
files.download('results.csv')

# Extension 1: Parameter Sensitivity

Explore multiple values for threshold to understand how different assumptions affect station coverage and the total range. A sensitivity analysis on theta can help us find an optimal balance.

## Vary Coverage Threshold

In [7]:
thetas = [0.1, 0.2, 0.3, 0.4, 0.5]
results = []

for theta in thetas:
    distance_data[['Coverage1', 'Coverage2']] = distance_data.apply(
        calculate_coverage,
        axis=1,
        average_ranges=average_range,
        theta=theta
    )
    C = calculate_C(distance_data)

    ########## Initialize Model ##########
    model = gb.Model(f"Extension 1: coverage threshold = {theta}")
    model.setParam('MIPGap', 1e-03)
    model.Params.OutputFlag = 0

    ########## Parameters ##########
    # Charging speeds in miles per hour
    charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
    environmental_impact = {2: 0.000027405, 3: 0.0002945}
    H = 10  # Available hours per day
    public_charger_demand = 0.3
    avg_percentage_charged = 0.6
    # Cost parameters (charger setup and station setup)
    cost_per_level2_charger = 8250
    cost_per_level3_charger = 65000
    # Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
    base_station_cost = 50000
    # Electrical upgrade calculations
    panel_capacity = 500 # amps
    usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
    existing_load = 200  # amps
    remaining_capacity = usable_capacity - existing_load  # amps
    upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers

    ########## Decision Variables ##########
    # Number of Level 2 and Level 3 chargers at each station
    x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
    x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
    z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
    y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")  # Binary variables for electrical upgrade tiers

    # Demand fulfilled by each charger type in miles
    demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
    demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")

    ########## Objectives ##########
    total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
    total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
    total_range = gb.quicksum((x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) for i in EV)
    # Calculate the environmental impact based on demand fulfilled by each type
    total_environmental_impact = (
        environmental_impact[2] * demand_fulfilled_by_level2 +
        environmental_impact[3] * demand_fulfilled_by_level3
    )
    #### Calculate the total cost
    M = 10000
    for i in EV:
        # Ensure only one tier is active for each location
        model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

        # Combined loads:
        combined_loads = x2[i] * 40 + x3[i] * 150

        # Constraints for each tier
        model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
        model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
        model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")

    electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

    station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
    chargers_setup_cost = gb.quicksum(
        x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
    )
    total_cost = chargers_setup_cost + station_setup_cost

    # 1. Minimize the total cost
    model.setObjectiveN(total_cost, index=0, priority=0)

    # 2. Minimize Environmental impact
    model.setObjectiveN(total_environmental_impact, index=1, priority=1)

    # 3. Maximize total electric range per hour of charging
    model.setObjectiveN(-total_range, index=2, priority=2)

    model.ModelSense = GRB.MINIMIZE

    ########## Constraints ##########
    # EV at any location can reach at least one of the charging stations
    for j in EV:
        model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

    # Demand Coverage with Charging Capacity Requirement
    for j in EV:
        total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
        model.addConstr(
            gb.quicksum(
                C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
                for i in EV
            ) >= total_demand_j,
            name=f"DemandCoverage_{j}"
        )

    # Calculate the demand fulfilled by each type of charger
    model.addConstr(
        demand_fulfilled_by_level2 == charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
        name="DemandFulfilledByLevel2"
    )
    model.addConstr(
        demand_fulfilled_by_level3 == charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
        name="DemandFulfilledByLevel3"
    )

    # Linking Charging Stations and Chargers
    M = 20  # Maximum number of charging stalls at each station
    for i in EV:
        model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
        model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

    model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
    model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
    model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)

    ########## Solve Model ##########
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        results.append({
            'theta': theta,
            'Total Charging Capacity per Hour': total_range.getValue(),
            'Total Environmental Impact': total_environmental_impact.getValue(),
            'Total Cost': total_cost.getValue(),
            'demand_fulfilled_level2': demand_fulfilled_by_level2.x,
            'demand_fulfilled_level3': demand_fulfilled_by_level3.x,
            'Total number of Level 2 Charger': sum(x2[i].x for i in EV),
            'Total number of Level 3 Charger': sum(x3[i].x for i in EV),
            'Total number of Station': sum(z[i].x for i in EV)
        })
    else:
        print("No optimal solution found.")

pd.DataFrame(results)

Set parameter MIPGap to value 0.001
Set parameter MIPGap to value 0.001
Set parameter MIPGap to value 0.001
Set parameter MIPGap to value 0.001
Set parameter MIPGap to value 0.001


,theta,Total Charging Capacity per Hour,Total Environmental Impact,Total Cost,demand_fulfilled_level2,demand_fulfilled_level3,Total number of Level 2 Charger,Total number of Level 3 Charger,Total number of Station
0,0.1,225270.0,481.28807,119939500.0,681900.0,1570800.0,4546.0,748.0,267.0
1,0.2,225270.0,481.28807,119899500.0,681900.0,1570800.0,4546.0,748.0,265.0
2,0.3,225270.0,481.28807,119949500.0,681900.0,1570800.0,4546.0,748.0,269.0
3,0.4,225270.0,481.28807,119889500.0,681900.0,1570800.0,4546.0,748.0,266.0
4,0.5,225270.0,481.28807,119904500.0,681900.0,1570800.0,4546.0,748.0,266.0


## Vary Average Percentage Charged

In [8]:
demand_scaling_factors = [0.5, 0.8, 1.0, 1.2, 1.5]
results = []

for scale in demand_scaling_factors:
    distance_data[['Coverage1', 'Coverage2']] = distance_data.apply(
        calculate_coverage,
        axis=1,
        average_ranges=average_range,
        theta=0.2
    )
    C = calculate_C(distance_data)

    ########## Initialize Model ##########
    model = gb.Model(f"Extension 1: percentage charged scaling = {scale}")
    model.setParam('MIPGap', 1e-03)
    model.Params.OutputFlag = 0

    ########## Parameters ##########
    # Charging speeds in miles per hour
    charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
    environmental_impact = {2: 0.000027405, 3: 0.0002945}
    H = 10  # Available hours per day
    public_charger_demand = 0.3
    avg_percentage_charged = 0.6 * scale
    # Cost parameters (charger setup and station setup)
    cost_per_level2_charger = 8250
    cost_per_level3_charger = 65000
    # Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
    base_station_cost = 50000
    # Electrical upgrade calculations
    panel_capacity = 500 # amps
    usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
    existing_load = 200  # amps
    remaining_capacity = usable_capacity - existing_load  # amps
    upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers

    ########## Decision Variables ##########
    # Number of Level 2 and Level 3 chargers at each station
    x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
    x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
    z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
    y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")  # Binary variables for electrical upgrade tiers

    # Demand fulfilled by each charger type in miles
    demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
    demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")

    ########## Objectives ##########
    total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
    total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
    total_range = gb.quicksum((x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) for i in EV)
    # Calculate the environmental impact based on demand fulfilled by each type
    total_environmental_impact = (
        environmental_impact[2] * demand_fulfilled_by_level2 +
        environmental_impact[3] * demand_fulfilled_by_level3
    )
    #### Calculate the total cost
    M = 10000
    for i in EV:
        # Ensure only one tier is active for each location
        model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

        # Combined loads:
        combined_loads = x2[i] * 40 + x3[i] * 150

        # Constraints for each tier
        model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
        model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
        model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")


    electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

    station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
    chargers_setup_cost = gb.quicksum(
        x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
    )
    total_cost = chargers_setup_cost + station_setup_cost

    # 1. Minimize the total cost
    model.setObjectiveN(total_cost, index=0, priority=0)

    # 2. Minimize Environmental impact
    model.setObjectiveN(total_environmental_impact, index=1, priority=1)

    # 3. Maximize total electric range per hour of charging
    model.setObjectiveN(-total_range, index=2, priority=2)

    model.ModelSense = GRB.MINIMIZE

    ########## Constraints ##########
    # EV at any location can reach at least one of the charging stations
    for j in EV:
        model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

    # Demand Coverage with Charging Capacity Requirement
    for j in EV:
        total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
        model.addConstr(
            gb.quicksum(
                C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
                for i in EV
            ) >= total_demand_j,
            name=f"DemandCoverage_{j}"
        )

    # Calculate the demand fulfilled by each type of charger
    model.addConstr(
        demand_fulfilled_by_level2 == charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
        name="DemandFulfilledByLevel2"
    )
    model.addConstr(
        demand_fulfilled_by_level3 == charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
        name="DemandFulfilledByLevel3"
    )

    # Linking Charging Stations and Chargers
    M = 20  # Maximum number of charging stalls at each station
    for i in EV:
        model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
        model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

    model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
    model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
    model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)

    ########## Solve Model ##########
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        results.append({
            'Avg Percentage Charged': avg_percentage_charged,
            'Total Charging Capacity per Hour': total_range.getValue(),
            'Total Environmental Impact': total_environmental_impact.getValue(),
            'Total Cost': total_cost.getValue(),
            'demand_fulfilled_level2': demand_fulfilled_by_level2.x,
            'demand_fulfilled_level3': demand_fulfilled_by_level3.x,
            'Total number of Level 2 Charger': sum(x2[i].x for i in EV),
            'Total number of Level 3 Charger': sum(x3[i].x for i in EV),
            'Total number of Station': sum(z[i].x for i in EV)
        })
    else:
        print("No optimal solution found.")

pd.DataFrame(results)

Set parameter MIPGap to value 0.001
Set parameter MIPGap to value 0.001
Set parameter MIPGap to value 0.001
Set parameter MIPGap to value 0.001
Set parameter MIPGap to value 0.001


,Avg Percentage Charged,Total Charging Capacity per Hour,Total Environmental Impact,Total Cost,demand_fulfilled_level2,demand_fulfilled_level3,Total number of Level 2 Charger,Total number of Level 3 Charger,Total number of Station
0,0.30,112635.0,149.778316,80993250.0,681150.0,445200.0,4541.0,212.0,238.0
1,0.48,180225.0,348.910994,104271750.0,680850.0,1121400.0,4539.0,534.0,255.0
2,0.60,225270.0,481.288070,119899500.0,681900.0,1570800.0,4546.0,748.0,265.0
3,0.72,270330.0,614.791054,135254500.0,678900.0,2024400.0,4526.0,964.0,275.0
4,0.90,337920.0,813.923733,158623000.0,678600.0,2700600.0,4524.0,1286.0,293.0


# Extension 2: Weighted Multi-Objective Function

We’ll replace the priority-based objective setup with a single objective function where each component has a weight. This allows more flexibility in balancing environmental impact, total charging capacity, and costs.

In [9]:
########## Initialize Model ##########
model = gb.Model("Extension 2: Weighted Multi-Objective Function")
model.setParam('MIPGap', 1e-03)
model.Params.OutputFlag = 0

########## Parameters ##########
# Charging speeds in miles per hour
charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
environmental_impact = {2: 0.000027405, 3: 0.0002945}
H = 10  # Available hours per day
public_charger_demand = 0.3
avg_percentage_charged = 0.6
# Cost parameters (charger setup and station setup)
cost_per_level2_charger = 8250
cost_per_level3_charger = 65000
# Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
base_station_cost = 50000
# Electrical upgrade calculations
panel_capacity = 500 # amps
usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
existing_load = 200  # amps
remaining_capacity = usable_capacity - existing_load  # amps
upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers

########## Decision Variables ##########
# Number of Level 2 and Level 3 chargers at each station
x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")  # Binary variables for electrical upgrade tiers

# Demand fulfilled by each charger type in miles
demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")

########## Constraints ##########
# EV at any location can reach at least one of the charging stations
for j in EV:
    model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

# Demand Coverage with Charging Capacity Requirement
for j in EV:
    total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
    model.addConstr(
        gb.quicksum(
            C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
            for i in EV
        ) >= total_demand_j,
        name=f"DemandCoverage_{j}"
    )

# Calculate the demand fulfilled by each type of charger
model.addConstr(
    demand_fulfilled_by_level2 == charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
    name="DemandFulfilledByLevel2"
)
model.addConstr(
    demand_fulfilled_by_level3 == charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
    name="DemandFulfilledByLevel3"
)

# Linking Charging Stations and Chargers
M = 20  # Maximum number of charging stalls at each station
for i in EV:
    model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
    model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)

########## Objectives ##########
total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
total_range = gb.quicksum((x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) for i in EV)
# Calculate the environmental impact based on demand fulfilled by each type
total_environmental_impact = (
    environmental_impact[2] * demand_fulfilled_by_level2 +
    environmental_impact[3] * demand_fulfilled_by_level3
)
#### Calculate the total cost
M = 10000
for i in EV:
    # Ensure only one tier is active for each location
    model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

    # Combined loads:
    combined_loads = x2[i] * 40 + x3[i] * 150

    # Constraints for each tier
    model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")


electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
chargers_setup_cost = gb.quicksum(
    x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
)
total_cost = chargers_setup_cost + station_setup_cost

########## Weighted Objective Function ##########
# Define weights for each component of the objective function
# These weights should be chosen based on the relative importance of each objective
# For example purposes, arbitrary weights are assigned. Adjust these as needed.
weight_total_range = 150000      # Prioritize range
weight_environmental_impact = 10000  # Moderate weight for sustainability
weight_total_cost = 0.000001     # Small weight, ensures cost isn't excessive

# Formulate the single objective function
single_objective = (
    weight_total_range * (-total_range) +              # Maximize total range
    weight_environmental_impact * total_environmental_impact +  # Minimize environmental impact
    weight_total_cost * total_cost                      # Minimize total cost
)

model.setObjective(single_objective, GRB.MINIMIZE)

########## Solve Model ##########
model.optimize()

if model.Status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Charging Capacity per Hour: {total_range.getValue()}")
    print(f'Total Environmental Impact: {total_environmental_impact.getValue()}')
    print(f'Total Cost: {total_cost.getValue()}')
    print(f"Total Demand Fulfilled by Level 2 Chargers (miles): {demand_fulfilled_by_level2.x}")
    print(f"Total Demand Fulfilled by Level 3 Chargers (miles): {demand_fulfilled_by_level3.x}")
    print(f'Total number of Level 2 Charger: {sum(x2[i].x for i in EV)}')
    print(f'Total number of Level 3 Charger: {sum(x3[i].x for i in EV)}')
    print(f'Total number of Station: {sum(z[i].x for i in EV)}')
    print("\n")
    for i in EV:
        if z[i].x > 0.5:  # Using 0.5 to account for numerical issues
            level2_chargers = int(x2[i].x)
            level3_chargers = int(x3[i].x)
            print(f"Set up station at {i} with {level2_chargers} Level 2 chargers and {level3_chargers} Level 3 chargers")
else:
    print("No optimal solution found.")

Set parameter MIPGap to value 0.001
Optimal Solution Found:
Total Charging Capacity per Hour: 338250.0
Total Environmental Impact: 819.062265
Total Cost: 163590000.0
Total Demand Fulfilled by Level 2 Chargers (miles): 663000.0
Total Demand Fulfilled by Level 3 Chargers (miles): 2719500.0
Total number of Level 2 Charger: 4420.0
Total number of Level 3 Charger: 1295.0
Total number of Station: 486.0


Set up station at 98188.0 with 20 Level 2 chargers and 0 Level 3 chargers
Set up station at 98052.0 with 5 Level 2 chargers and 0 Level 3 chargers
Set up station at 98115.0 with 5 Level 2 chargers and 0 Level 3 chargers
Set up station at 98033.0 with 5 Level 2 chargers and 0 Level 3 chargers
Set up station at 98006.0 with 5 Level 2 chargers and 0 Level 3 chargers
Set up station at 98072.0 with 5 Level 2 chargers and 0 Level 3 chargers
Set up station at 98040.0 with 5 Level 2 chargers and 0 Level 3 chargers
Set up station at 98103.0 with 5 Level 2 chargers and 0 Level 3 chargers
Set up statio

# Extension 3: Charger Breakdowns

In real life, chargers do not work a 100% of the times. Our original model assumes a 100% reliability of chargers. To make it more realistic, we will introduce a new factor called the reliability factor that would give us how reliable a certain type of charger is.

In [17]:
# Initialize Model
model = gb.Model("EV_Charging_Optimization_Reliability")
model.setParam('MIPGap', 1e-03)
model.Params.OutputFlag = 0


################################## Parameters ##################################

# Charging speeds in miles per hour
charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
reliability_factor = {2: 0.95, 3: 0.90}
effective_charging_speed = {
    level: charging_speed[level] * reliability_factor[level] for level in charging_speed
}

environmental_impact = {2: 0.000027405, 3: 0.0002945}
H = 10  # Available hours per day
public_charger_demand = 0.3
avg_percentage_charged = 0.6
peak_charging_ev = 0.1

# Cost parameters
cost_per_level2_charger = 8250
cost_per_level3_charger = 65000
# Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
base_station_cost = 50000
# Electrical upgrade calculations
panel_capacity = 500 # amps
usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
existing_load = 200  # amps
remaining_capacity = usable_capacity - existing_load  # amps
upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers


############################## Decision Variables ##############################

x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")

demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")


################################## Objectives ##################################

total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
total_range = gb.quicksum((x2[i] * effective_charging_speed[2] + x3[i] * effective_charging_speed[3]) for i in EV)

# Calculate the environmental impact based on demand fulfilled by each type
total_environmental_impact = (
    environmental_impact[2] * demand_fulfilled_by_level2 +
    environmental_impact[3] * demand_fulfilled_by_level3
)
#### Calculate the total cost
M = 10000
for i in EV:
    # Ensure only one tier is active for each location
    model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

    # Combined loads:
    combined_loads = x2[i] * 40 + x3[i] * 150

    # Constraints for each tier
    model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")

electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
chargers_setup_cost = gb.quicksum(
    x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
)
total_cost = chargers_setup_cost + station_setup_cost

# 1. Minimize the total cost
model.setObjectiveN(total_cost, index=0, priority=0)

# 2. Minimize Environmental impact
model.setObjectiveN(total_environmental_impact, index=1, priority=1)

# 3. Maximize total electric range per hour of charging
model.setObjectiveN(-total_range, index=2, priority=2)

model.ModelSense = GRB.MINIMIZE


################################## Constraints #################################
# EV at any location can reach at least one of the charging stations
for j in EV:
    model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

# Demand Coverage with Charging Capacity Requirement
for j in EV:
    total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
    model.addConstr(
        gb.quicksum(
            C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
            for i in EV
        ) >= total_demand_j,
        name=f"DemandCoverage_{j}"
    )

# Calculate the demand fulfilled by each type of charger
model.addConstr(
    demand_fulfilled_by_level2 == effective_charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
    name="DemandFulfilledByLevel2"
)
model.addConstr(
    demand_fulfilled_by_level3 == effective_charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
    name="DemandFulfilledByLevel3"
)

# Linking Charging Stations and Chargers
M = 20  # Maximum number of charging stalls at each station
for i in EV:
    model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
    model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)


# Solve the Model
model.optimize()

# Output Results
if model.Status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Charging Capacity per Hour: {total_range.getValue()}")
    print(f'Total Environmental Impact: {total_environmental_impact.getValue()}')
    print(f'Total Cost: {total_cost.getValue()}')
    print(f"Total Demand Fulfilled by Level 2 Chargers (miles): {demand_fulfilled_by_level2.x}")
    print(f"Total Demand Fulfilled by Level 3 Chargers (miles): {demand_fulfilled_by_level3.x}")
    print(f'Total number of Level 2 Charger: {sum(x2[i].x for i in EV)}')
    print(f'Total number of Level 3 Charger: {sum(x3[i].x for i in EV)}')
    print(f'Total number of Station: {sum(z[i].x for i in EV)}')
    print("\n")
    for i in EV:
        if z[i].x > 0.5:  # Using 0.5 to account for numerical issues
            level2_chargers = int(x2[i].x)
            level3_chargers = int(x3[i].x)
            print(f"Set up station at {i} with {level2_chargers} Level 2 chargers and {level3_chargers} Level 3 chargers")
else:
    print("No optimal solution found.")


Set parameter MIPGap to value 0.001
Optimal Solution Found:
Total Charging Capacity per Hour: 225288.0
Total Environmental Impact: 490.8282939
Total Cost: 127142000.0
Total Demand Fulfilled by Level 2 Chargers (miles): 646380.0
Total Demand Fulfilled by Level 3 Chargers (miles): 1606500.0
Total number of Level 2 Charger: 4536.0
Total number of Level 3 Charger: 850.0
Total number of Station: 271.0


Set up station at 98006.0 with 20 Level 2 chargers and 0 Level 3 chargers
Set up station at 98103.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98117.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98021.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98053.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98607.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98133.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up station at 98155.0 with 19 Level 2 chargers and 1 Level 3 chargers
Set up

# Extension 4: Traveling Maintenance Man

Someone needs to maintain these chargers. A maintaince man is hired to travel between the stations and repair them.




In [11]:
distance_data = pd.read_csv('distance.csv')

# Extract the first 10 unique zipcodes
zipcodes = list(distance_data['PostalCode1'].unique()[:10])

# Create a dictionary to represent the graph
graph = {}
for zipcode in zipcodes:
    graph[zipcode] = {}

# Populate the graph with distances, converting to integers
for index, row in distance_data.iterrows():
    zip1 = row['PostalCode1']
    zip2 = row['PostalCode2']
    distance = int(row['Distance'])  # Convert distance to integer

    if zip1 in zipcodes and zip2 in zipcodes and distance > 0:
        graph.setdefault(zip1, {})[zip2] = distance
        graph.setdefault(zip2, {})[zip1] = distance


graph

{98122.0: {98109.0: 3,
  98027.0: 20,
  98597.0: 72,
  98903.0: 169,
  98012.0: 23,
  98366.0: 24,
  98001.0: 28,
  98144.0: 2,
  98110.0: 15},
 98109.0: {98122.0: 3,
  98027.0: 24,
  98597.0: 73,
  98903.0: 172,
  98012.0: 22,
  98366.0: 22,
  98001.0: 30,
  98144.0: 5,
  98110.0: 12},
 98027.0: {98122.0: 20,
  98109.0: 24,
  98597.0: 73,
  98903.0: 149,
  98012.0: 31,
  98366.0: 42,
  98001.0: 26,
  98144.0: 19,
  98110.0: 35},
 98597.0: {98122.0: 72,
  98109.0: 73,
  98027.0: 73,
  98903.0: 156,
  98012.0: 95,
  98366.0: 61,
  98001.0: 47,
  98144.0: 69,
  98110.0: 71},
 98903.0: {98122.0: 169,
  98109.0: 172,
  98027.0: 149,
  98597.0: 156,
  98012.0: 178,
  98366.0: 183,
  98001.0: 151,
  98144.0: 167,
  98110.0: 182},
 98012.0: {98122.0: 23,
  98109.0: 22,
  98027.0: 31,
  98597.0: 95,
  98903.0: 178,
  98366.0: 43,
  98001.0: 50,
  98144.0: 26,
  98110.0: 30},
 98366.0: {98122.0: 24,
  98109.0: 22,
  98027.0: 42,
  98597.0: 61,
  98903.0: 183,
  98012.0: 43,
  98001.0: 32,
  981

In [12]:
# Create the Gurobi model
tsp_model = gb.Model("Traveling_Salesman_Problem")
tsp_model.Params.OutputFlag = 0  # Suppress Gurobi output

# Decision variables: x[i,j] = 1 if we travel from city i to city j
x = tsp_model.addVars(zipcodes, zipcodes, vtype=GRB.BINARY, name="x")


# Objective function: Minimize total travel distance
tsp_model.setObjective(
    gb.quicksum(graph[i][j] * x[i, j] for i in zipcodes for j in zipcodes if i != j), GRB.MINIMIZE
)


# Constraints
# 1. Each city must be visited exactly once
for city in zipcodes:
    tsp_model.addConstr(sum(x[city, j] for j in zipcodes if j != city) == 1, f"out_{city}")
    tsp_model.addConstr(sum(x[i, city] for i in zipcodes if i != city) == 1, f"in_{city}")


#2. Subtour elimination (Miller-Tucker-Zemlin formulation)
u = tsp_model.addVars(zipcodes, vtype=GRB.INTEGER, lb=1, ub=len(zipcodes), name="u")

for i in zipcodes[1:]:  # Start from the second city to avoid conflicts with the starting point
    for j in zipcodes[1:]:
        if i != j:
            tsp_model.addConstr(
                u[i] - u[j] + len(zipcodes) * x[i, j] <= len(zipcodes) - 1, f"subtour_{i}_{j}"
            )


# Optimize the model
tsp_model.optimize()


if tsp_model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Distance: {tsp_model.ObjVal}")

    # Reconstruct the tour
    tour = []
    visited = set()
    current_city = zipcodes[0]
    while len(tour) < len(zipcodes):
        tour.append(current_city)
        visited.add(current_city)
        for next_city in zipcodes:
            if next_city not in visited and x[current_city, next_city].x > 0.5:
                current_city = next_city
                break
    # Close the loop
    tour.append(tour[0])
    print(" -> ".join(map(str, tour)))

Optimal Solution Found:
Total Distance: 469.0
98122.0 -> 98012.0 -> 98027.0 -> 98903.0 -> 98597.0 -> 98001.0 -> 98366.0 -> 98110.0 -> 98109.0 -> 98144.0 -> 98122.0
